# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [3]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Вопрос аналитику
Какие оценки поставил пользователь №1 и в каком количестве?

Что сделаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [3]:
ratings[ratings.userId == 1].rating.value_counts()

2.0    7
3.0    4
2.5    3
4.0    3
1.0    2
3.5    1
Name: rating, dtype: int64

In [4]:
ratings.groupby(['userId','rating']).count().head(6)

movieId  timestamp
userId rating                    
1      1.0           2          2
       2.0           7          7
       2.5           3          3
       3.0           4          4
       3.5           1          1
       4.0           3          3

Как это сделать для всех пользователей сразу:

In [5]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head(10)


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5
6,1,2,3,7,0,7,3,11,6,4
7,0,3,0,5,0,41,0,26,0,13
8,1,0,0,3,4,13,26,35,16,18
9,0,0,0,5,0,9,0,23,0,8


In [6]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


### Фильтры и вычисления с помощью loc и iloc

In [9]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [10]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


In [15]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

,visit_id,region
0,e3b0c44298,Russia
1,6e340b9cff,Russia
2,96a296d224,Russia
3,709e80c884,Russia
4,df3f619804,Russia


In [11]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [17]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2


In [18]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,1.2


### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [19]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [20]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Adventure
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN


Можно и через loc

In [21]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

,userId,movieId,rating,timestamp,title,genres,Adventure
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,NaN


Для замера времени сделаем эти операции для набор жанров

In [22]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [23]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)

CPU times: total: 2.7 s
Wall time: 2.78 s


In [24]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']

CPU times: total: 109 ms
Wall time: 156 ms


### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [30]:
log['traffic_type'] = log.apply(lambda row: 'organic' if ('yandex' or 'google') in row.traffic_source else 'Nan', axis=1)
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,Nan
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,organic


In [39]:
log2 = pd.read_csv('visit_log.csv', sep=';')

In [32]:
log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,VAT,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2,Nan
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2,organic


In [46]:
log2.loc[(log2.traffic_source == 'yandex') | (log2.traffic_source == 'google'), 'traffic_type'] = 'organic' 
log2['traffic_type'].value_counts()

organic    5407
Name: traffic_type, dtype: int64

**###Решил добавить еще один вариант решения через отдельную функцию.**

In [12]:
%%time
import pandas as pd
data = pd.read_csv('visit_log.csv', sep =';')
traffic_type1 = []
def traffic_type():
    for word in data['traffic_source']:
        if word == 'yandex':
            traffic_type1.append('organic')
        elif word == 'google':
            traffic_type1.append('organic')
        else:
            traffic_type1.append('NaN')
traffic_type()
data['traffic_type'] = traffic_type1
data['traffic_type'].value_counts()

CPU times: total: 15.6 ms
Wall time: 20.5 ms


NaN        13531
organic     5407
Name: traffic_type, dtype: int64

### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [1]:
import numpy as np

In [6]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


In [7]:
# фильтр на пустые значения в столбце value

df.loc[~pd.isnull(df.value), :]

,value
0,123.0
5,456.0


In [8]:
df.fillna(0)

,value
0,123.0
1,0.0
2,0.0
3,0.0
4,0.0
5,456.0


In [9]:
m = df.fillna(0).value.mean()
m

96.5

In [10]:
df.fillna(m)

,value
0,123.0
1,96.5
2,96.5
3,96.5
4,96.5
5,456.0


In [11]:
df.value.mean()

289.5

### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

In [93]:
log = pd.read_csv('visit_log.csv', sep=';')

In [94]:
log.loc[log.traffic_source.isin(['yandex', 'google']), 'traffic_type'] = 'organic'
log.head(7)

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,NaN


In [100]:
log.loc[pd.isnull(log.traffic_type), :] = log.loc[pd.isnull(log.traffic_type), :].fillna('other')
log.head(7)

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,other


In [18]:
%%time
import pandas as pd
data = pd.read_csv('visit_log.csv', sep =';')
traffic_type1 = []
def traffic_type():
    for word in data['traffic_source']:
        if word == 'yandex':
            traffic_type1.append('organic')
        elif word == 'google':
            traffic_type1.append('organic')
        else:
            traffic_type1.append('other')
traffic_type()
data['traffic_type'] = traffic_type1
data['traffic_type'].value_counts()

CPU times: total: 15.6 ms
Wall time: 19.1 ms


other      13531
organic     5407
Name: traffic_type, dtype: int64

### Дата и время в pandas

In [53]:
log = pd.read_csv('visit_log.csv', sep=';')

In [54]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [55]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,2019-02-12 14:12:16


In [56]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 7 columns):
timestamp         18938 non-null int64
visit_id          18938 non-null object
url               18938 non-null object
region            18938 non-null object
user_id           18938 non-null object
traffic_source    18938 non-null object
date              18938 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 1.0+ MB


In [57]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,date,hour
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,2019-02-12 14:11:32,14
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,2019-02-12 14:11:44,14
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,2019-02-12 14:11:55,14
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,2019-02-12 14:12:05,14
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,2019-02-12 14:12:16,14


### Методы работы со строками

In [58]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


Проверка наличия подстроки в строке в питоне:

In [59]:
'охотник' in 'каждый охотник желает знать...'

True

Аналог в pandas:

In [60]:
stats[stats.keyword.str.contains('охотник')].head()

,keyword,shows
3072,сумеречные охотники 2 сезон,71965
3474,сумеречные охотники,66083
3654,белоснежка и охотник 2 фильм 2016,63154
4178,последний охотник на ведьм,57560
6127,последний охотник на ведьм фильм 2015,42213


[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [61]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

,keyword,shows
20,фильмы 2016,4486635
51,фильмы,2156641
54,фильмы онлайн,2305540
68,смотреть фильмы онлайн,1928484
86,порно фильмы,1458031


### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

In [54]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [61]:
stats[stats.keyword.str.contains('погода в моск|погода в новосибирс|погода в краснодар')].value_counts()

keyword                                       shows 
гисметео погода в москве на 2 недели          23149     1
нгс погода в новосибирске на 14 дней          5936      1
погода в новосибирске на неделю               9098      1
погода в новосибирске на 3 дня                15475     1
погода в новосибирске на 14 дней              93278     1
погода в новосибирске на 14                   3785      1
погода в новосибирске на 10 дней              10778     1
погода в новосибирске                         133362    1
погода в москве сейчас                        11353     1
погода в москве сегодня                       15647     1
погода в москве на сегодня                    9969      1
погода в москве на неделю от гидрометцентра   10359     1
погода в москве на неделю                     180096    1
погода в москве на месяц                      43801     1
погода в москве на завтра                     20354     1
погода в москве на 3 дня                      115394    1
погода в москве на 

### replace

In [62]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

'отпуск начнется через месяц'

Аналог в pandas на запросах про сериалы:

In [63]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

,keyword,shows
246,сериалы,587779
304,сериалы тут,503967
555,турецкие сериалы на русском языке,296403
881,русские сериалы,234262
890,сериалы онлайн,204812


In [64]:
serial.keyword.str.replace('сериалы', 'книги').head()

246                              книги
304                          книги тут
555    турецкие книги на русском языке
881                      русские книги
890                       книги онлайн
Name: keyword, dtype: object

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [65]:
serial.keyword.str.upper().head()

246                              СЕРИАЛЫ
304                          СЕРИАЛЫ ТУТ
555    ТУРЕЦКИЕ СЕРИАЛЫ НА РУССКОМ ЯЗЫКЕ
881                      РУССКИЕ СЕРИАЛЫ
890                       СЕРИАЛЫ ОНЛАЙН
Name: keyword, dtype: object

In [66]:
serial.keyword.str.lower().head()

246                              сериалы
304                          сериалы тут
555    турецкие сериалы на русском языке
881                      русские сериалы
890                       сериалы онлайн
Name: keyword, dtype: object

### Что делать если нужно учесть формы написания слов?

In [67]:
stats[stats.keyword.str.contains('рубл')].head()

,keyword,shows
1202,алиэкспресс на русском в рублях официальный сайт,161553
1602,курс гривны к рублю,125076
1736,али экспресс русская версия на русском в рубля...,117260
2132,доллары в рубли,114173
2172,100 долларов в рублях,97534


Почему нельзя просто оставить str.contains('рубл'):

In [68]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [69]:
from pymystem3 import Mystem

In [70]:
search = 'курс гривны к рублю рубли рублях'

In [71]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 '\n']

In [72]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль \n'

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [109]:
from pymystem3 import Mystem
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

In [110]:
new_list = []
for word in data['keyword']:
    word.split(sep = ' ')
    new_list.append(word)

In [111]:
%%time
new_list1 = []
new_list2 = []
new_list3 = []
for word in new_list:
    lemmas = m.lemmatize(word)
    new_list1.append(lemmas)
for list in new_list1:
    new_list2 = ' '.join(list)
    new_list3.append(new_list2)
data['lemmas'] = new_list3

CPU times: total: 0 ns
Wall time: 2.3 s


In [112]:
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль \n
1,доллары в рубли,114173,доллар в рубль \n
2,100 долларов в рублях,97534,100 доллар в рубль \n
3,курс рубля,53546,курс рубль \n
